In [ ]:
import pip
pip.main(['install', 'rembg'])

In [ ]:
#imports
import cv2
import numpy as np
import os
import random
import matplotlib.pyplot as plt
from skimage import  data, color, feature
import pandas as pd
from sklearn import svm
from sklearn.decomposition import PCA

# from rembg import remove

In [ ]:

def resize_image(image):
    return cv2.resize(image, (300, 300), interpolation=cv2.INTER_AREA)

def apply_gaussian_blur(image):
    return cv2.GaussianBlur(image, (5, 5), 0)

def get_skin_mask_hsv(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_skin = (0, 20, 70)
    upper_skin = (20, 255, 255)
    return cv2.inRange(hsv_image, lower_skin, upper_skin)

def get_skin_mask_ycrcb(image):
    ycrcb_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    lower_skin = (0, 135, 85)
    upper_skin = (255, 180, 135)
    return cv2.inRange(ycrcb_image, lower_skin, upper_skin)

def equalize_histogram(image):
    ycrcb_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    channels = cv2.split(ycrcb_image)
    channels[0] = cv2.equalizeHist(channels[0])
    return cv2.cvtColor(cv2.merge(channels), cv2.COLOR_YCrCb2BGR)

def preprocess_image(image):
    resized_image = resize_image(image)
    blurred_image = apply_gaussian_blur(resized_image)
    
    hsv_mask = get_skin_mask_hsv(blurred_image)
    ycrcb_mask = get_skin_mask_ycrcb(blurred_image)
    
    skin_mask = cv2.bitwise_and(hsv_mask, ycrcb_mask)
    
    # Apply morphological operations to improve the skin mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    skin_mask = cv2.erode(skin_mask, kernel, iterations=2)
    skin_mask = cv2.dilate(skin_mask, kernel, iterations=2)
    
    # Apply the skin mask to the original image
    skin_segment = cv2.bitwise_and(blurred_image, blurred_image, mask=skin_mask)
    
    # Convert the skin segment to grayscale and apply thresholding
    gray_image = cv2.cvtColor(skin_segment, cv2.COLOR_BGR2GRAY)
    ret, thresh_image = cv2.threshold(gray_image, 175, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return thresh_image



In [ ]:
data_train=[]
def test_stages(channel, index):
    # Load a sample images from the dataset

    sample_image_path = 'dataset/men/' + channel + '/' + channel + '_men (' + str(index) + ').JPG'
    img = cv2.imread(sample_image_path)
    
    # Preprocess the image
    img = cv2.imread(sample_image_path, -1)

    rgb_planes = cv2.split(img)

    result_planes = []
    result_norm_planes = []
    skin_mask=""
    skeleton=""
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((3,3), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 25)
        diff_img = np.max(dilated_img) - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img,None, alpha=170, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_planes.append(diff_img)
        result_norm_planes.append(norm_img)
    result = cv2.merge(result_planes)
    result_norm = color.rgb2gray(cv2.merge(result_norm_planes))

    cv2.imwrite('shadows_out.png', result)
    cv2.imwrite('shadows_out_norm.png', result_norm)
    hogVec, hogVis = feature.hog(result_norm, visualize=True)
    data_train.append(hogVec.reshape(hogVec.shape[0],1))
    return data_train
# Test the stages of the preprocessing pipeline by atterating over the channel with 5 indexes
data_train_4=[]
data_train_2=[]
data_train_3=[]
for index in range(1, 5):
   data_train_4=test_stages(str(4), str(index))
data_train=[]
for index in range(1, 6):
   data_train_2=test_stages(str(2), str(index))
data_train=[]
for index in range(1, 6):
   data_train_3=test_stages(str(3), str(index))


In [ ]:
X=[]
Y=[]
obj_shape=data_train_4[0].shape
X=[data_train_4[0].reshape(obj_shape[1],obj_shape[0]),data_train_2[0].reshape(obj_shape[1],obj_shape[0]),data_train_4[2].reshape(obj_shape[1],obj_shape[0]),data_train_4[1].reshape(obj_shape[1],obj_shape[0]),data_train_2[3].reshape(obj_shape[1],obj_shape[0]),data_train_2[2].reshape(obj_shape[1],obj_shape[0]),data_train_2[1].reshape(obj_shape[1],obj_shape[0]),data_train_3[0].reshape(obj_shape[1],obj_shape[0]),data_train_3[3].reshape(obj_shape[1],obj_shape[0])]
Y=[4,2,4,4,2,2,2,3,3]
print(np.array(X).shape,X[0].shape,len(X))

In [ ]:

#will put the features in the dataframe
print(obj_shape[0])
df=pd.DataFrame(columns=[ i for i in range(14971068)])
print(df)

In [ ]:
for i in range(len(X)):
    x_flatten=(X[i].flatten()).reshape(-1)
    df.append([x_flatten]) 
print(df)

In [ ]:
#to save a 95% from the usefull features
pca=PCA(.95)
x_pca=pca.fit_transfrom(df)
print(x_pca)

In [ ]:
# split the data to train and test data
from sklearn .model_selection import train_test_split
X_train,x_test,y_train,y_test=train_test_split(x_pca,Y,test_size=0.2,random_state=30)

In [ ]:
#create a svm model 
clf = svm.SVC(kernel='linear')
#fit the data to the model
clf.fit(X_train,y_train)